In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Computing Total Derivatives

Problem has a method, `compute_totals`, that allows you to compute the unscaled total derivative values for a model.

If the model approximated its Jacobian, the method uses an approximation method.

```{eval-rst}
    .. automethod:: openmdao.core.problem.Problem.compute_totals
        :noindex:
```

## Usage

Here is a simple example of using `compute_totals`:

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src64", get_code("openmdao.test_suite.components.paraboloid.Paraboloid"), display=False)

:::{Admonition} `Paraboloid` class definition 
:class: dropdown

{glue:}`code_src64`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid())

model.set_input_defaults('comp.x', 3.0)
model.set_input_defaults('comp.y', -4.0)

prob.setup()

prob.run_model()

totals = prob.compute_totals(of=['comp.f_xy'], wrt=['comp.x', 'comp.y'])
print(totals[('comp.f_xy', 'comp.x')][0][0])

In [ ]:
print(totals[('comp.f_xy', 'comp.y')][0][0])

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
import numpy as np

assert_near_equal(totals[('comp.f_xy', 'comp.x')][0][0], -4.0, tolerance=1e-8)
assert_near_equal(totals[('comp.f_xy', 'comp.y')][0][0], 3.0, tolerance=1e-8)

In [ ]:
totals = prob.compute_totals(of=['comp.f_xy'], wrt=['comp.x', 'comp.y'], return_format='dict')
print(totals['comp.f_xy']['comp.x'][0][0])

In [ ]:
print(totals['comp.f_xy']['comp.y'][0][0])

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
import numpy as np

assert_near_equal(totals['comp.f_xy']['comp.x'][0][0], -4.0, tolerance=1e-8)
assert_near_equal(totals['comp.f_xy']['comp.y'][0][0], 3.0, tolerance=1e-8)

By default, `compute_totals` returns the derivatives unscaled, but you can also request that they be scaled by the driver scale values declared when the des_vars, objectives, or constraints are added:

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid())

model.set_input_defaults('comp.x', 3.0)
model.set_input_defaults('comp.y', -4.0)

model.add_design_var('comp.x', 3.0, ref0=50.0)
model.add_design_var('comp.y', -4.0)
model.add_objective('comp.f_xy')

prob.setup()

prob.run_model()

totals = prob.compute_totals(of=['comp.f_xy'], wrt=['comp.x', 'comp.y'], driver_scaling=True)
print(totals[('comp.f_xy', 'comp.x')][0][0])

In [ ]:
print(totals[('comp.f_xy', 'comp.y')][0][0])

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
import numpy as np

assert_near_equal(totals[('comp.f_xy', 'comp.y')][0][0], 3.0, tolerance=1e-8)